In [0]:
mode = 'mt-dnn_assin+tweetsent' #'st-dnn', 'mt-dnn_assin+tweetsent'
pretrained = 'bert-pt'

output_dir = f'report/{mode}/{pretrained}'

filepath = \
f'output/{mode}/tweesent/{pretrained}/tweetsent_test_scores_4.json' if mode == 'st-dnn' \
else  f'output/{mode}/{pretrained}/tweetsent_test_scores_4.json'

Get results

In [0]:
import pandas as pd
import json

with open(filepath) as f:
    output = json.load(f)

rows = list()
headers = ['id', 'prediction']
labels = ['Negative', 'Neutral', 'Positive']

for idx, answer in enumerate(output['predictions']):
    uid = output['uids'][idx]
    label = labels[answer]
    
    rows.append((uid, label)) 

result = pd.DataFrame(rows, columns = headers)

Extract dataset

In [8]:
!unzip tweetSentBR_extracted.zip

Archive:  tweetSentBR_extracted.zip
   creating: tweetSentBR_extracted/
  inflating: tweetSentBR_extracted/testTT.neg  
  inflating: tweetSentBR_extracted/testTT.neu  
  inflating: tweetSentBR_extracted/testTT.pos  
  inflating: tweetSentBR_extracted/trainTT.neg  
  inflating: tweetSentBR_extracted/trainTT.neu  
  inflating: tweetSentBR_extracted/trainTT.pos  
  inflating: tweetSentBR_extracted/tweets.none  
  inflating: tweetSentBR_extracted/tweets.neg  
  inflating: tweetSentBR_extracted/tweets.neu  
  inflating: tweetSentBR_extracted/tweets.pos  


Get structered dataset

In [36]:
from io import StringIO
import pandas as pd
import os
import re

corpora = [f for f in os.listdir('tweetSentBR_extracted') if 'testTT' in f]

print('Separating lines in tabs...')
tabbed = dict()
for goldpath in corpora:
    with open("tweetSentBR_extracted/{}".format(goldpath)) as f:
        text = f.read()

    assert '\t' not in text 
    
    outtext = re.sub(r'(.+?) (.+)',r'\1\t\2', text)
    tabbed.update({goldpath:StringIO(outtext)})
    print(goldpath)
    
print('Unifying..')

header = ['id', 'premise']
abbr = {'neg': 'Negative', 'neu': 'Neutral', 'pos': 'Positive'} 
corpus = pd.DataFrame()

for path, f in tabbed.items():
    table = pd.read_csv(f, sep = '\t', names = header)

    posfix = path.split('.')[1]
    label = abbr[posfix]
    table['label'] = label

    corpus = table if corpus.empty else corpus.append(table)

Separating lines in tabs...
testTT.pos
testTT.neu
testTT.neg
Unifying..


Genarate and save prediction list

In [0]:
number = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

predictions = list()
i = 0
for corpus_idx, corpus_row in corpus.iterrows():
    uid = corpus_row['id']
    corpus_id_type = corpus['id'].dtype
    result_row  = result.loc[result['id'].astype(corpus_id_type) == uid]
    prediction = result_row.iloc[0]['prediction']
    predictions.append(number[prediction])

import json
with open(f'{output_dir}/tweetsent.json', 'w') as f:
   json.dump(predictions, f) 